In [1]:
import sys
print(sys.version)

3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]


In [2]:
pip install datasets kaggle PyMuPDF Pillow pytesseract opencv-python 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\annap\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\annap\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
#RUN THIS COMMAND ON TERMINAL --> TO CONNECT TO KAGGLE  OR RUN HERE WITH OS
import os

os.system(r'icacls "C:\Users\annap\.kaggle\kaggle.json" /inheritance:r /grant:r annap:R')


0

In [18]:
import kaggle
kaggle.api.authenticate()

from PIL import Image
import pytesseract
import pandas as pd
import numpy as np
import os
import cv2
import re

# from transformers import AutoProcessor, AutoModelForSequenceClassification, TrainingArguments, Trainer
from huggingface_hub import hf_hub_download
import torch
from transformers import LayoutLMv3ForSequenceClassification, LayoutLMv3Tokenizer
from datasets import Dataset  
import subprocess


## From JPG


In [10]:
dataset_path = 'shaz13/real-world-documents-collections'
kaggle.api.dataset_download_files(dataset_path, path=dataset_path, unzip=True)
print(os.listdir(dataset_path))


Dataset URL: https://www.kaggle.com/datasets/shaz13/real-world-documents-collections
['docs-sm']


In [11]:
np.random.seed(500)
#invoices
invoice_path = f'{dataset_path}/docs-sm/invoice'
print(os.listdir(invoice_path))


['0000137486.jpg', '0000192502.jpg', '0000226471.jpg', '0000227351.jpg', '0000333206.jpg', '0000542607.jpg', '0001136788.jpg', '0001219743.jpg', '0001219844.jpg', '0001233629.jpg', '0001453135.jpg', '0011519293.jpg', '0011929750.jpg', '0011929754.jpg', '0011930170.jpg', '0011991025.jpg', '0012179618.jpg', '0012193483.jpg', '0012287015.jpg', '0013041942.jpg', '0013043203.jpg', '0013122156.jpg', '0013267193.jpg', '0060026309.jpg', '0060027381.jpg', '0060031784.jpg', '0060032039.jpg', '0060046962.jpg', '0060051594.jpg', '0060075043.jpg', '0060080760.jpg', '0060082098.jpg', '0060083495.jpg', '0060095587.jpg', '0060095589.jpg', '0060116328.jpg', '0060121960.jpg', '0060201834.jpg', '0060207574.jpg', '0060281129.jpg', '0060284223.jpg', '00621985.jpg', '00920638.jpg', '00920717.jpg', '01145176.jpg', '01398469.jpg', '03638969.jpg', '03660266.jpg', '03700650.jpg', '03724966.jpg', '03750991.jpg', '1000031431.jpg', '10399076.jpg', '10399232.jpg', '11235021_11235022.jpg', '11238840.jpg', '200120567

# Utility Functions for LayoutMv3

In [12]:
def normalize_box(bbox, w, h):
    return [
        int(bbox[0]*(1000/w)),
        int(bbox[1]*(1000/h)),
        int(bbox[2]*(1000/w)),
        int(bbox[3]*(1000/h)),
    ]


def unnormalize_box(bbox, w, h):
    return [
        w * (bbox[0] / 1000),
        hash * (bbox[1] / 1000),
        w * (bbox[2] / 1000),
        h * (bbox[3] / 1000),
    ]


#comparing bounding boxes for debugging
def compare_boxes(b1, b2):
    b1 = np.array([c for c in b1])
    b2 = np.array([c for c in b2])
    equal = np.array_equal(b1, b2)
    return equal


def adjacent(w1, w2):
    if w1['label'] == w2['label'] and abs(w1['id'] - w2['id']) == 1:
      return True
    return False



# Preprocessing

In [ ]:
'''tesseract runs OCR on images to recognize text'''
def run_tesseract_on_image(image_path):  # -> tsv output path
    # image_path = image_path.replace("\\", "/") 

    base_path = r"C:\Users\annap\Desktop\Year3\stats\document-classification" 
    image_path = os.path.join(base_path, image_path)  
    image_path = os.path.normpath(image_path) 
    print("IMAGE PATH", image_path)

    # image_path = r"C:\Users\annap\Desktop\Year3\stats\final_project\"" + image_path
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    print("image name after supposedly removing the .tsv", image_name)

    #DEBUG
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Unable to read image: {image_path}")   
    else:
        print(f"Image {image_path} loaded successfully.")  


    tsv_directory = os.path.abspath(os.path.join(os.getcwd(), "tsv"))
    if not os.path.exists(tsv_directory):
        os.makedirs(tsv_directory)

    tsv_output_path = os.path.join(tsv_directory, f"{image_name}.tsv")
    tsv_output_path = os.path.normpath(tsv_output_path)

    print(f"Running Tesseract on image: {image_path}")
    print(f"Saving TSV to: {tsv_output_path}")
    print(f"Command: tesseract \"{image_path}\" \"{tsv_output_path}\" -l eng tsv")


    tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    # error_code = os.system(f'"{tesseract_cmd}" "{image_path}" "{tsv_output_path}" -l eng tsv')
    result = subprocess.run([tesseract_cmd, image_path, tsv_output_path, "-l", "eng", "tsv"], capture_output=True, text=True)
    

    # if not error_code:
    if result.returncode != 0:
        print(f"Error running Tesseract: {result.stderr}")
        raise ValueError(f"Error running Tesseract on image {image_path}. Check Tesseract logs.")
    else:
        print(f"Tesseract completed successfully. Output: {result.stdout}")
        return tsv_output_path



'''reads the file generated above, removes empty/missing entries,
iterates through the cleaned dataset and stores words and bbox coordinates'''
def clean_tesseract_output(tsv_output_path):
    ocr_df = pd.read_csv(tsv_output_path, sep='\t')
    ocr_df = ocr_df.dropna()
    ocr_df = ocr_df.drop(ocr_df[ocr_df.text.str.strip() == ''].index)
    # text_output = ' '.join(ocr_df.text.tolist())

    words = []
    for index, row in ocr_df.iterrows():
      word = {}
      origin_box = [row['left'], row['top'], row['left'] + row['width'], row['top']+row['height']]
      word['word_text'] = row['text']
      word['word_box'] = origin_box
      words.append(word)

    return words          #array of dicts that include bbox coords and words details




In [33]:
def extract_text_bboxes(image_path, use_tesseract_tsv=True):
    if use_tesseract_tsv:                     #tesseract TSV
      tsv_path = run_tesseract_on_image(image_path)
      words_boxes = clean_tesseract_output(tsv_path)
      words = [item['word_text'] for item in words_boxes]
      bboxes = [item['word_box'] for item in words_boxes]

    # else:                                     #pytesseract
    #   img = cv2.imread(image_path)
    #   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #   gray = cv2.GaussianBlur(gray, (5,5), 0)
    #   _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #   extracted_text = pytesseract.image_to_string(thresh)

    #   data = pytesseract.image_to_data(thresh, output_type=pytesseract.Output.DICT)   #extracts words, bboxes

    #   words, bboxes = [], []
    #   h, w, _ = img.shape

    #   for i in range(len(data["text"])):
    #     if data["text"][i].strip():
    #       words.append(data["text"][i])
    #       x,y,w,h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
    #       bbox = [
    #           int((x/w)*1000),
    #           int((y/h)*1000),
    #           int(((x + w)/w) * 1000),
    #           int(((y + h)/h) *1000)]      #1000x1000  normalizes bboxes
    #       bboxes.append(bbox)

    return words, bboxes



def clean_text(df):
    # df = df.dropna()
    df = df.explode("tokens")
    df = df[df["tokens"].str.strip() != '']

    # text_output = ' '.join(df.text.tolist())

    words = []
    for index, row in df.iterrows():
        word = {}
        origin_box = row['bboxes']
        word['word_text'] = row['tokens']
        word['word_box'] = origin_box
        words.append(word)

    return words



#prepares for inference
invoice_data_list = []
print(invoice_path)

for filename in os.listdir(invoice_path):
    if filename.lower().endswith(('.jpg', '.jpeg',)):
        print(f"Valid image: {filename}")
        image_path = os.path.join(invoice_path, filename)
        print(f"Full image path before calling fucntion: {image_path}")
        if os.path.exists(image_path):
            print(f"Image exists: {image_path}")
        else:
            print(f"Image does NOT exist: {image_path}")
            
        words,bboxes = extract_text_bboxes(image_path, use_tesseract_tsv=True)
        # invoice_data = extract_invoice_data(text)
        # invoice_data["File Name"] = filename
        invoice_data_list.append({
            "file_name": filename,
            "tokens": words,
            "bboxes": bboxes,
        })
    else:
        print(f"Skipping non-image file: {filename}")

df = pd.DataFrame(invoice_data_list)
# print(df)
# print(df.columns)


df = clean_text(df)
print(df)


shaz13/real-world-documents-collections/docs-sm/invoice
Valid image: 0000137486.jpg
Full image path before calling fucntion: shaz13/real-world-documents-collections/docs-sm/invoice\0000137486.jpg
Image exists: shaz13/real-world-documents-collections/docs-sm/invoice\0000137486.jpg
IMAGE PATH C:\Users\annap\Desktop\Year3\stats\document-classification\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg
image name after supposedly removing the .tsv 0000137486
Image C:\Users\annap\Desktop\Year3\stats\document-classification\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg loaded successfully.
Running Tesseract on image: C:\Users\annap\Desktop\Year3\stats\document-classification\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg
Saving TSV to: c:\Users\annap\Desktop\Year3\stats\document-classification\tsv\0000137486
Command: tesseract "C:\Users\annap\Desktop\Year3\stats\document-classification\shaz13\real-world-documents-collection

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\annap\\Desktop\\Year3\\stats\\document-classification\\tsv\\0000137486'

LayoutMv3 needs textual and spatial information. So, we need the bounding boxes of where the text is positioned within the picture.

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

#convert into huggnig face dataset
hf_dataset = Dataset.from_pandas(df)
print(hf_dataset[0])   #check it worked


#passes words/bboxes into the layoutmv3 processer
def preprocess(example, img_path):
  img = Image.open(image_path.convert("RBG"))
  encoding = processor(
      images=img,
      text=example["tokens"],
      boxes=example["bboxes"],
      truncation=True,
      padding="max_length"
  )
  return encoding


dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


# Invoice Extraction

In [ ]:
# we will need to recreate this for all categories
def extract_invoice_data(text):
    data = {}

    data["Invoice Number"] = re.search(r'Invoice\s*No[:#]?\s*([\w-]+)', text, re.IGNORECASE)
    data["Invoice Date"] = re.search(r'Invoice\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Due Date"] = re.search(r'Due\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Issuer Name"] = re.search(r'From[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Recipient Name"] = re.search(r'To[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Total Amount"] = re.search(r'Total\s*Amount[:#]?\s*([\d,]+\.?\d{2})', text, re.IGNORECASE)

    for key, value in data.items():
        data[key] = value.group(1) if value else "Not Found"

    return data


# LayoutMv3

In [ ]:
#pretrained layoutmv3 model
model = AutoModelForSequenceClassification("microsoft/layoutlmv3-base", num_labels=4)

train_args = TrainingArguments(output_dir="./layoutlmv3_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs"
    )


trainer= Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
    )


trainer.train()

